In [1]:
library(bnlearn)
library(forecast)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [36]:
dag = read.table("../results/dag.prediction.train.response.warp.directed.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=T)

data_test = read.table("../data/test.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=T)

data_numeric = read.table("../data/prediction.train.numeric.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=F)
data_response = read.table("../data/prediction.train.response.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=F)
data_binary = read.table("../data/prediction.train.binary.warp.txt", header = TRUE, sep="\t", row.names=1, check.names=F, stringsAsFactors=T)

In [37]:
e = empty.graph(colnames(dag))

In [38]:
data_numeric <- as.data.frame(lapply(data_numeric, as.numeric))
data_response <- as.data.frame(lapply(data_response, as.numeric))

data_train = cbind(data_numeric, data_binary, data_response)

In [39]:
amat(e) = as.matrix(dag)

In [6]:
# data = dedup(data_numeric, threshold=.95)

In [40]:
data_train <- data_train[colnames(data_train) %in% colnames(dag)]
data_test <- data_test[colnames(data_test) %in% colnames(dag)]

In [41]:
data_test$StudyDayNo <- as.numeric(data_test$StudyDayNo)

In [42]:
length(unique(colnames(dag)))

[1] 223

In [43]:
data_train

,StudyDayNo,tax.k__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium.s__Bifidobacterium_bifidum,tax.k__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium.s__Bifidobacterium_pseudocatenulatum,tax.k__Bacteria.p__Actinobacteriota.c__Coriobacteriia.o__Coriobacteriales.f__Coriobacteriaceae.g__Collinsella.s__Collinsella_sp003466125,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Prevotella.s__Prevotella_sp003447235,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Bacteroidaceae.g__Prevotella.s__Prevotella_sp900313215,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Barnesiellaceae.g__Barnesiella.s__Barnesiella_sp003150885,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Rikenellaceae.g__Alistipes_A.s__Alistipes_A_ihumii,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Rikenellaceae.g__Alistipes_A.s__Alistipes_A_indistinctus,tax.k__Bacteria.p__Bacteroidota.c__Bacteroidia.o__Bacteroidales.f__Rikenellaceae.g__Alistipes_A.s__Alistipes_A_sp900240235,⋯,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__RF32.f__CAG_239.g__CAG_267.s__CAG_267_sp001917135,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__RF32.f__CAG_239.g__CAG_495.s__CAG_495_sp000436375,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__RF32.f__CAG_239.g__CAG_495.s__CAG_495_sp001917125,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Alphaproteobacteria.o__RF32.f__CAG_239.g__CAG_495.s__CAG_495_sp900540425,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales.f__Burkholderiaceae.g__Mesosutterella.s__Mesosutterella_massiliensis,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales.f__Burkholderiaceae.g__Sutterella.s__Sutterella_wadsworthensis,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Burkholderiales.f__Burkholderiaceae.g__Sutterella.s__Sutterella_wadsworthensis_A,day_plus_one.tax.k__Bacteria.p__Proteobacteria.c__Gammaproteobacteria.o__Enterobacterales.f__Enterobacteriaceae.g__Escherichia.s__Escherichia_coli,day_plus_one.tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila,day_plus_one.tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila_B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,3,1.8815860,6.266098,0.09353664,-12.47795737,-12.47796,-12.4779574,2.8469065,-1.2932607,-12.4803669,⋯,0.838473796,-12.908055994,-12.908056,-12.9082473,-12.908056,-12.908056,-12.907110,0.35400819,0.98113565,5.99300846
3,4,-0.1141617,3.398695,-12.90454897,-12.90805599,-12.90806,-12.9080560,4.3277427,-12.9041970,-1.7284775,⋯,1.384347201,-12.937864827,-12.937865,-12.1827562,-12.937865,-12.937865,-16.672041,-0.80088430,4.40697117,5.17444560
4,5,-10.0145946,2.489555,-15.60301073,-12.93786483,-12.93786,-12.9378648,4.6474165,-18.4660707,-4.8858497,⋯,0.773744223,-12.711781844,-12.711782,-12.7104224,-12.711782,-12.711782,-12.718505,0.07862995,0.08709675,4.35651547
5,6,-12.7157614,3.114358,-12.71255732,-12.71178184,-12.71178,-12.7117818,4.3563585,-12.7182422,-12.7098724,⋯,-0.847428842,-12.722656823,-12.722657,-12.7271810,-12.722657,-12.722657,-1.539104,2.17210882,-12.72338570,4.69906426
6,7,-0.4472806,3.371808,-12.72284839,-12.72265682,-12.72266,-12.7226568,4.6756737,-0.4458144,-12.7220734,⋯,0.939899892,-13.129479960,-13.129480,-1.2606222,-13.129480,-13.129480,-13.127925,1.45474470,0.12459399,5.09008417
7,8,0.1327485,3.239770,-13.12760962,-13.12947996,-13.12948,-13.1294800,4.4642538,-1.9457325,-13.1345897,⋯,0.543258291,-13.212362833,-13.212363,-2.0229787,-13.212363,-13.212363,-2.

In [44]:
length(unique(colnames(data_train)))

[1] 223

In [45]:
fitted = bn.fit(e, data_train)

In [46]:
# pred = predict(fitted, "day_plus_one.tax.k__Bacteria.p__Actinobacteriota.c__Actinomycetia.o__Actinomycetales.f__Bifidobacteriaceae.g__Bifidobacterium.s__Bifidobacterium_adolescentis", data_test)

columns = colnames(data_test)[startsWith(colnames(data_test), "day_plus_one")]
baseline_columns = replace(columns, "day_plus_one.", "")

results <- data.frame(matrix(ncol = 4, nrow = 0))
colnames(results) <- c("baseline", "model", "truth")
for (col in columns) {
    pred = predict(fitted, col, data_test)
    temp_df <- cbind(as.vector(data_test[gsub("day_plus_one.", "", col)]), pred, data_test[, col], col)
    colnames(temp_df) <- c("baseline", "model", "truth", "taxon")
    results <- rbind(results, temp_df)
}

In [47]:
write.table(results, "../data/results.dbn.warp.txt", sep="\t")

In [ ]:
# columns = colnames(data_train)[startsWith(colnames(data_train), "day_plus_one")]
# baseline_columns = replace(columns, "day_plus_one.", "")

# results <- data.frame(matrix(ncol = 3, nrow = 0))
# colnames(results) <- c("baseline", "model", "truth")
# for (col in columns) {
#     pred = predict(fitted, col, data_train)
#     temp_df <- cbind(as.vector(data_train[gsub("day_plus_one.", "", col)]), pred, data_train[, col])
#     colnames(temp_df) <- c("baseline", "model", "truth")
#     results <- rbind(results, temp_df)
# }

In [31]:
accuracy(f = results$baseline, x = results$truth)

Warning message in accuracy.default(f = results$baseline, x = results$truth):
“Using `f` as the argument for `accuracy()` is deprecated. Please use `object` instead.”


,ME,RMSE,MAE,MPE,MAPE
Test set,-0.02106823,4.94318,2.810504,-214.5098,643.6203


In [32]:
accuracy(f = results$model, x = results$truth)

Warning message in accuracy.default(f = results$model, x = results$truth):
“Using `f` as the argument for `accuracy()` is deprecated. Please use `object` instead.”


,ME,RMSE,MAE,MPE,MAPE
Test set,0.0009483143,4.606742,3.110603,-116.9975,589.921


In [ ]:
summary(lm(baseline-model~1, results))

In [ ]:
t.test(results$baselne, results$model)

In [ ]:
plot(results$model, results$baseline)